In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel
from public_datasets_game.bayes_opt import BayesOptGame
from public_datasets_game.mechanism import PrivateFunding


env = BayesOptGame(
    num_consumers=3, mechanism=PrivateFunding(agent_budget_per_collector_step=2)
)
action = {agent: env.action_space(agent).sample() for agent in env.agents}
env.step(action)[0]

kernel = ConstantKernel() * RBF(length_scale=1.0, length_scale_bounds=(1e-1, 1e2))
consumer = env.consumers[0]


In [ ]:
print(action)

In [ ]:
# collected = np.random.rand(consumer.X.shape[0]) > 0.8
collected = consumer.collected

collected_points = consumer.X[collected == 1]
collected_values = consumer.Y[collected == 1]

print(collected.sum())

gpr = GaussianProcessRegressor(kernel=kernel)

gpr.fit(collected_points, collected_values)
pred_y = gpr.predict(consumer.X)

pred_y = pred_y.reshape(50, 50)
real_y = consumer.Y.reshape(50, 50)

fig, axs = plt.subplots(1, 2)
axs[0].imshow(real_y)
axs[1].imshow(pred_y)
